# Creating a cluster/slice

## Please read carefully all the instructions. Specially the comments given in each cell. These comments are pertinent to run a cluster successfully. 

If the fabric environment is not set please read through the topic <code>FABRIC Environment Setup</code> in <code>start_here.ipynb</code> notebook given under <code>jupyter-example</code> folder in fabric's JupyterHub.

<b> At first read the comments in the cell and then only run the cell. </b>

## STEP-1

In [18]:
# Run this cell.

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,68926660-da26-475d-9c40-50ebf0a5a812
Bastion Host,bastion.fabric-testbed.net
Bastion Username,ajaykumar_0000102567
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


In [89]:
# Run this cell
# To find all the available resources at this time
#run the cell below for easier readability 
try:
    resources = fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

Name,State,Address,Location,PTP Capable,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,Ram Available,Ram Capacity,Ram Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,P4-Switch Available,P4-Switch Capacity,P4-Switch Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated,A30 Available,A30 Capacity,A30 Allocated,A40 Available,A40 Capacity,A40 Allocated,U280 Available,U280 Capacity,U280 Allocated
UTAH,Active,"875 South West Temple,Salt Lake City, UT 84101","(40.7503666, -111.893838)",True,5,10,422,640,218,1638,2390,752,101721,107461,5740,571,635,64,1,1,0,2,2,0,4,4,0,16,16,0,4,4,0,1,6,5,0,0,0,0,0,0,1,1,0
HAWI,Active,"2520 Correa Road,Honolulu, Hi 96822","(21.2989762, -157.8163991)",True,5,10,508,640,132,1910,2390,480,103403,107463,4060,606,635,29,0,0,0,2,2,0,4,4,0,16,16,0,0,0,0,0,0,0,8,8,0,0,0,0,1,1,0
TACC,Active,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261807)",False,5,10,346,640,294,1542,2390,848,103023,107463,4440,593,635,42,0,0,0,2,2,0,4,4,0,16,16,0,4,4,0,3,6,3,0,0,0,0,0,0,1,1,0
GPN,Active,"5115 Oak Street,Kansas City,MO 64112","(39.0342627, -94.5826075)",False,5,10,520,640,120,1590,2390,800,54375,61455,7080,594,635,41,0,0,0,2,2,0,4,4,0,12,16,4,0,4,4,6,6,0,0,0,0,0,0,0,1,1,0
FIU,Active,"11001 SW 14th St,Miami,FL 33199","(25.7542948, -80.3702894)",True,5,10,464,640,176,1702,2390,688,50193,58713,8520,606,635,29,1,1,0,2,2,0,4,4,0,11,16,5,1,4,3,0,6,6,0,0,0,0,0,0,1,1,0
INDI,Active,"535 West Michigan Street,Indianapolis, IN 46202","(39.7737312, -86.1674868)",False,3,6,255,384,129,1112,1434,322,51968,56618,4650,299,381,82,0,0,0,1,2,1,2,2,0,10,10,0,0,0,0,0,0,0,4,4,0,0,0,0,1,1,0
MASS,Active,"100 Bigelow Street,Holyoke, MA 01040","(42.202493, -72.6078766)",True,3,6,322,384,62,1218,1434,216,61848,64068,2220,358,381,23,0,0,0,2,2,0,2,2,0,10,10,0,2,2,0,1,3,2,0,0,0,0,0,0,1,1,0
NEWY,Active,"32 Sixth Avenue,New York, NY 10013","(40.7383575, -73.9992012)",False,2,4,172,256,84,684,956,272,2096,4466,2370,221,254,33,0,0,0,3,3,0,2,3,1,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SALT,Active,"572 Delong Street,Salt Lake City, UT 84104","(40.7570751, -111.9534664)",False,3,6,304,384,80,1210,1434,224,61861,64161,2300,358,381,23,0,0,0,2,2,0,2,2,0,10,10,0,2,2,0,2,3,1,0,0,0,0,0,0,0,1,1
MAX,Active,"4161 Fieldhouse Drive,College Park,MD 20742","(38.9886345, -76.9434794)",True,5,10,540,640,100,1974,2390,416,103823,107463,3640,607,635,28,0,0,0,2,2,0,4,4,0,16,16,0,4,4,0,3,6,3,0,0,0,0,0,0,1,1,0


In [90]:
##for easier readability of available resources
import pandas as pd 
pd.DataFrame(resources.data)[['Hosts','CPUs','Name','Cores Available','Ram Available','Tesla T4 Available','RTX6000 Available']]


,Hosts,CPUs,Name,Cores Available,Ram Available,Tesla T4 Available,RTX6000 Available
0,5,10,UTAH,422,1638,4,1
1,5,10,HAWI,508,1910,0,0
2,5,10,TACC,346,1542,4,3
3,5,10,GPN,520,1590,0,6
4,5,10,FIU,464,1702,1,0
5,3,6,INDI,255,1112,0,0
6,3,6,MASS,322,1218,2,1
7,2,4,NEWY,172,684,0,0
8,3,6,SALT,304,1210,2,2
9,5,10,MAX,540,1974,4,3


## Step-2: Creating a cluster from an instance type.

In [81]:
# Run this cell
# Initialize the variables appropriately. 
# Number of nodes in the cluster.
num_nodes=6

# Give a cluster name
slice_name='TACC_GPU'

# Make it True if you want to include persistence storage to a single node. KEEP IT FALSE FOR THE TIME BEING.
storage=False

# get attached to the cluster. By default adding NVMe is false.
add_NVMe = True

# Make it True if you want to include GPUs to different nodes in the cluster [depends on availability].
add_gpu=True

# By default master node will not have GPUs in it. For CPU only cluster False means lower resources then the workers.
master_gpu=False

# If only t4 gpu needs to be added.
add_t4=True

# If only rtx gpu needs to be added.
add_rtx=True

# Site name, pick one site from the above list of resources.
site='TACC'

# Select node resource:- It follows the pattern fabric.c#N.m#N.d#N , where c:cores, m:primary memory, d:disk and #N: capacity.
# eg: fabric.c4.m8.d50 means cores:4, Memory:8Gb, disk:50Gb  
# List of node instance type is provided in - https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json
#instance_worker='fabric.c24.m128.d500'
instance_worker='fabric.c8.m64.d500'

# The resources of master node. It is better to be less than the worker nodes.
instance_master='fabric.c24.m64.d1000'

# Operation system, Linux distribution e.g. default_ubuntu_18, default_ubuntu_20, etc.
image='default_ubuntu_20'


In [82]:
import pandas as pd

try:
    json_format=fablib.list_sites(output='json',quiet=True)
except Exception as e:
    print(f"Exception: {e}")

sites_df=pd.read_json(json_format)
site_df=sites_df[sites_df['name']==site]
type_t4=site_df['tesla_t4_available'].values
type_rtx=site_df['rtx6000_available'].values
nvme_available=site_df['nvme_available'].values
print("Number of Nvidia t4 available now at",site,"is:", type_t4[0])
print("Number of Nvidia rtx6000 available now at",site,"is:", type_rtx[0])
print("Number of NVMe available now at",site,"is:", nvme_available[0])

max_type_t4 = type_t4[0]
max_type_rtx = type_rtx[0]
total_gpus = max_type_t4 + max_type_rtx

Number of Nvidia t4 available now at TACC is: 4
Number of Nvidia rtx6000 available now at TACC is: 3
Number of NVMe available now at TACC is: 16


/tmp/ipykernel_160/1190618741.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sites_df=pd.read_json(json_format)


In [83]:
# The max available GPU for the chosen site is mentioned above, to change it please make changes to the variables below.
# By default initialized with the available GPUs for the site.

custom_number_of_t4 = max_type_t4
custom_number_of_rtx = max_type_rtx

# custom_number_of_t4 = 3
# custom_number_of_rtx =4

if custom_number_of_t4 < max_type_t4 or custom_number_of_rtx < max_type_rtx:
    max_type_t4 = custom_number_of_t4
    max_type_rtx = custom_number_of_rtx
    total_gpus = max_type_t4 + max_type_rtx
print(total_gpus)
gpu_names=[]

for i in range(1,total_gpus+1):
    gpu_names.append("GPU_{0}".format(i))
 
print(gpu_names)
temp_t4=max_type_t4
temp_rtx=max_type_rtx

######################################################
#####                  NVMe                      #####
######################################################

# Assumption: If the number of available NVMes' are less than the number of cluster node minus the master NVMes' will not

if nvme_available >= num_nodes-1 and add_NVMe == True:
    add_NVMe = True
print(add_NVMe)

7
['GPU_1', 'GPU_2', 'GPU_3', 'GPU_4', 'GPU_5', 'GPU_6', 'GPU_7']
True


In [84]:
# Run this cell

node_names=[]
nic_names=[]
iface_names=[]
nvme_names=[]
network_name='cluster_network'
storage_name = 'gaf-storage'

for i in range(1,num_nodes+1):
    node_names.append("Node{0}".format(i))
    nic_names.append("Nic{0}".format(i))
    iface_names.append("iface{0}".format(i))
    nvme_names.append("nvme{0}".format(i))

print(node_names)
print(nic_names)
print(iface_names)
print(nvme_names)

['Node1', 'Node2', 'Node3', 'Node4', 'Node5', 'Node6']
['Nic1', 'Nic2', 'Nic3', 'Nic4', 'Nic5', 'Nic6']
['iface1', 'iface2', 'iface3', 'iface4', 'iface5', 'iface6']
['nvme1', 'nvme2', 'nvme3', 'nvme4', 'nvme5', 'nvme6']


In [85]:
# Only run when persistence storage needed to be attached. 
if storage == True:
    import traceback
    from plugins import Plugins
    try:
        Plugins.load()
    except Exception as e:
        traceback.print_exc()
print(temp_t4)
print(temp_rtx)

4
3


In [86]:
# Run this cell, Visit the link below to find different instance type options. 
# Read the comments carefully given below and make changes as necessary. 

try:
    slice=fablib.new_slice(name=slice_name)
    
    for i in range(num_nodes):
        if master_gpu == False and i == 0:
            node=slice.add_node(name=node_names[i],
                                site=site, 
                                instance_type=instance_master,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
        
        else:
            node=slice.add_node(name=node_names[i],
                                site=site,
                                instance_type=instance_worker,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]
            
            if add_gpu == True:
                print(i)
                if add_t4 == True and temp_t4 > 1 and i==1: # allocating 2 T4
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-2])
                    temp_t4 = temp_t4 - 2 
                    print("allocated 2T4")
                if add_t4 == True and temp_t4 > 0 and i==2: # allocating 1 T4
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
                    temp_t4 = temp_t4 - 1 
                    print("allocated 1T4")
                if add_rtx == True and temp_rtx >2 and i==3: # allocating 3rtx
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-3])
                    temp_rtx = temp_rtx - 3
                    print("allocated 3rtx")
                if add_rtx == True and temp_rtx >1 and i==4: # allocating 2 T4
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
                    temp_rtx = temp_rtx - 2
                    print("allocated 2rtx")
                if add_rtx == True and temp_rtx >0 and i==5: # allocating 1 rtx
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    temp_rtx = temp_rtx - 1
                    print("allocated 1rtx")
                else: 
                    print("No GPUs available")
            if add_NVMe == True:
                node.add_component(model='NVME_P4510', name=nvme_names[i-1])
except Exception as e:
    print(f'Exception: {e}') 

1
allocated 2T4
No GPUs available
2
allocated 1T4
No GPUs available
3
allocated 3rtx
No GPUs available
4
No GPUs available
5
No GPUs available


In [87]:
# Run this cell
try:
    net_cluster=slice.add_l2network(name=network_name, interfaces=iface_names[:])
except Exception as e:
    print(f"Exception: {e}")

In [88]:
# Run this cell
# If this cell get executed successfully then IP address of the nodes will be displayed which can be used to ssh into the respective nodes.
# If there is an error while creating the slice/cluster, repeat from the third code cell block.
try:
    slice.submit()
except Exception as e:
    print(f'Exception: {e}')
    


Retry: 1, Time: 47 sec


ID,c883d34b-e0ec-4bf1-b8ed-b87622fe15ec
Name,TACC_GPU
Lease Expiration (UTC),2025-01-04 21:28:56 +0000
Lease Start (UTC),2025-01-03 21:28:56 +0000
Project ID,68926660-da26-475d-9c40-50ebf0a5a812
State,Closing


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
6b3c4858-459f-40e5-acbf-3709432d4026,Node1,24,64,1000,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93acbb8c-ba39-49d1-a14b-cf6ce8ece10a,Node2,8,64,500,default_ubuntu_20,qcow2,tacc-w4.fabric-testbed.net,TACC,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b9a5538c-67e9-467a-a3f5-dee53f229289,Node3,None,None,None,default_ubuntu_20,qcow2,None,TACC,ubuntu,,Closed,Insufficient resources : Component of type: Tesla T4 not available in graph node: 8QNDZC3#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d27234b0-3392-4fbe-9fe4-4678c1975175,Node4,8,64,500,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
050ad6e9-735f-4cff-aaf7-092d18b98db8,Node5,8,64,500,default_ubuntu_20,qcow2,tacc-w4.fabric-testbed.net,TACC,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8e12e735-a65e-42d8-99db-3b7f974a0717,Node6,8,64,500,default_ubuntu_20,qcow2,tacc-w4.fabric-testbed.net,TACC,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
615cc84e-0c05-4c22-a47d-c71bfa57b69e,cluster_network,L2,L2Bridge,TACC,cluster_network.subnet,cluster_network.gateway,Closed,"redeem predecessor reservation# b9a5538c-67e9-467a-a3f5-dee53f229289 is in a terminal state, failing the reservation# 615cc84e-0c05-4c22-a47d-c71bfa57b69e#"


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-Node1-p1,p1,Node1,cluster_network,100,config,,,,,,6,p1
Node2-Node2-p1,p1,Node2,cluster_network,100,config,,,,,,4,p1
Node3-Node3-p1,p1,Node3,cluster_network,100,config,,,,,,None,p1
Node4-Node4-p1,p1,Node4,cluster_network,100,config,,,,,,6,p1
Node5-Node5-p1,p1,Node5,cluster_network,100,config,,,,,,4,p1
Node6-Node6-p1,p1,Node6,cluster_network,100,config,,,,,,4,p1



Time to print interfaces 48 seconds


## Step-3: Configuring the network and setting up the cluster

In [68]:
# try:
#     slice=fablib.get_slice(name="GPU-Variant-Calling") # Put the cluster name that you want to delete
#     #slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")

In [96]:
#Run this cell
from ipaddress import IPv4Address, IPv4Network
try:
    subnet=IPv4Network("192.168.1.0/24")
    available_ips=list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [70]:
#try:
#    for node in slice.get_nodes():
#        node_iface=node.get_interface(network_name=network_name)
#        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
#except Exception as e:
#    print(f"Exception: {e}")

In [97]:
#Run this cell
#%%capture
try:
    for node in slice.get_nodes():
        node_iface=node.get_interface(network_name=network_name)
        node_IP_addr=available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_IP_addr, subnet=subnet)
        
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        _, _ = node.execute('sudo apt-get update')
        stdout, stderr = node.execute('sudo apt install net-tools')        
        stdout, stderr = node.execute(f'sudo ifconfig {node_iface.get_os_interface()} up')
        
        
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 46:10:44:d1:b4:a4 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::4410:44ff:fed1:b4a4/64 scope link 
       valid_lft forever preferred_lft forever
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3304 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [484 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata

In [98]:
#Run this cell
# If the ping is successful, that means the nodes are connected properly if there is any error then may need to recreate the cluster or further have to look into it. 
try:
    node1=slice.get_node(name='Node1')
    stdout,stderr=node1.execute(f' ping -c 3 192.168.1.4')
    print(stdout)
    print(stderr)
except Exception as e:
    print(f'Exception: {e}')


PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
From 192.168.1.1 icmp_seq=1 Destination Host Unreachable
From 192.168.1.1 icmp_seq=2 Destination Host Unreachable
From 192.168.1.1 icmp_seq=3 Destination Host Unreachable

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 0 received, +3 errors, 100% packet loss, time 2051ms
pipe 3
PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
From 192.168.1.1 icmp_seq=1 Destination Host Unreachable
From 192.168.1.1 icmp_seq=2 Destination Host Unreachable
From 192.168.1.1 icmp_seq=3 Destination Host Unreachable

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 0 received, +3 errors, 100% packet loss, time 2051ms
pipe 3




In [99]:
# Run this cell
# Function to create a file that contains IPs and hostnames related to it.

def append_line(file_path,text):
    with open(file_path,"a+") as file_des:
        file_des.seek(0)
        data=file_des.read(-1)
        if len(data)>0:
            file_des.write("\n")
        file_des.write(text)

In [100]:
# Run this cell

import os

if os.path.isfile('/home/fabric/work/hosts') or os.path.isfile('/home/fabric/work/ips.txt') or os.path.isfile('/home/fabric/work/workers'):
    os.system("rm /home/fabric/work/hosts")
    os.system("rm /home/fabric/work/ips.txt")
    os.system("rm /home/fabric/work/workers")    
else:
    print("does not exist")

if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
    os.system("rm /home/fabric/work/gpu_ips.txt")
    
#os.system("rm /home/fabric/work/ips.txt")

rm: cannot remove '/home/fabric/work/ips.txt': No such file or directory
rm: cannot remove '/home/fabric/work/workers': No such file or directory


In [101]:
# Run this cell

# Capturing the IP addresses, this step needs to be integrated with the IP assigning stage, coded above. Or it may stay independent.
import os

i=1
local_host="127.0.0.1 localhost"
path_to_host_file="/home/fabric/work/hosts"
path_to_ip_file="/home/fabric/work/ips.txt"
path_to_worker_ip="/home/fabric/work/workers"
path_to_gpu_ips="/home/fabric/work/gpu_ips.txt"
gpu_name="NVIDIA"

append_line(path_to_host_file,local_host)

try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute("hostname -I")
        IP=stdout.split(" ")[1]
        node_name="node{0}".format(i)
        vm_names="vm{0}".format(i-1)
        append_line(path_to_ip_file,IP)
        
        stdout, stderr=node.execute("hostname")
        line=IP+" "+node_name+" "+vm_names+" "+stdout
        append_line(path_to_host_file,line)
      
    
        if(i>1):
            append_line(path_to_worker_ip,vm_names)
            #append_line(path_to_worker_ip,IP)
        
        print(line)
        print(stderr)
        
        stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
        if gpu_name in stdout_gpu:
            stdout_ip, _=node.execute('hostname -I')
            ip=stdout_ip.split(" ")[1]
            append_line(path_to_gpu_ips,ip)
            #print(ip)
        
        i=i+1
except Exception as e:
    print(f"Exception: {e}")
    
print(IP)

10.30.6.89 192.168.1.1 2620:103:a006:12:f816:3eff:fe94:ac37 
Node1
192.168.1.1 node1 vm0 Node1


10.30.6.104 192.168.1.2 2620:103:a006:12:f816:3eff:fe0e:db56 
Node2
192.168.1.2 node2 vm1 Node2


09:00.0 3D controller: NVIDIA Corporation TU104GL [Tesla T4] (rev a1)
10.30.6.104 192.168.1.2 2620:103:a006:12:f816:3eff:fe0e:db56 
192.168.1.2


In [102]:
# Run this cell

try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') # if you run the command twice the back up will be overwritten, a conditional block should be written
        output_host_copy=node.upload_file(path_to_host_file,"/home/ubuntu/hosts")
        output_ip_copy=node.upload_file(path_to_ip_file,"/home/ubuntu/ips.txt")
        output_worker_copy=node.upload_file(path_to_worker_ip,"/home/ubuntu/workers")
        stdout_host_copy,stderr_host_copy=node.execute(f'sudo cp /home/ubuntu/hosts /etc/hosts')
        if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
            output_gpu_copy=node.upload_file(path_to_gpu_ips,"/home/ubuntu/gpu_ips.txt")
        
        print(stderr)
        print(stderr_host_copy)
except Exception as e:
    print(f"Exception : {e}")
    

In [103]:
# Run this cell

import os

output=os.system('ssh-keygen -q -t rsa -N "" -f /home/fabric/work/id_rsa > /dev/null 2>&1')

In [104]:
# Run this cell

try:
    for node in slice.get_nodes():
        output_private=node.upload_file("/home/fabric/work/id_rsa","/home/ubuntu/.ssh/id_rsa")
        output_public=node.upload_file("/home/fabric/work/id_rsa.pub","/home/ubuntu/.ssh/id_rsa.pub")
        
        stdout, stderr=node.execute(f' cat /home/ubuntu/.ssh/id_rsa.pub >> /home/ubuntu/.ssh/authorized_keys')
        stdout, stderr=node.execute(f' chmod 600 /home/ubuntu/.ssh/id_rsa*')
        
        print(output_private)
        print(output_public)
        print(stderr)
        #print(stdout)
        
except Exception as e:
    print(f"Exception: {e}")

-rw-rw-r--   1 1000     1000         2643 03 Dec 17:13 ?
-rw-rw-r--   1 1000     1000          598 03 Dec 17:13 ?

-rw-rw-r--   1 1000     1000         2643 03 Dec 17:13 ?
-rw-rw-r--   1 1000     1000          598 03 Dec 17:13 ?



In [105]:
# Run this cell. This is the last cell to run. Please read the comments in the next few cells to know how to extend the lease time and how to delete a slice/cluster. 

from ipaddress import ip_address, IPv6Address
try:
    for node in slice.get_nodes():
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            node.upload_file("/home/fabric/work/nat64.sh", "/home/ubuntu/nat64.sh")
            #stdout, stderr=node.execute(f' chmod +x /home/ubuntu/nat64.sh && sudo bash /home/ubuntu/nat64.sh')
            stdout, stderr=node.execute(f' chmod +x /home/ubuntu/nat64.sh && sudo bash /home/ubuntu/nat64.sh > /dev/null 2>&1')
            print(stdout)
            print(stderr)
except Exception as e:
    print(f"Exception: {e}")
    

## Step-4: Deleting and extending lease of a slice

In [ ]:
# # Run this cell ONLY when you want to delete the cell
# # To delete a slice/cluster.
# try:
#     slice=fablib.get_slice(name="AVAH") # Put the cluster name that you want to delete
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")

In [ ]:
# Run this cell ONLY when you want to exted the time of the slice. Or to extend the lease time of the slice/cluster.

import datetime
slice_name='demo' # Give the cluster/slice name that you want to extend

#Set end host to now plus 1 day
new_end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
#new_end_date = (datetime.now(timezone.utc) + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
print(type(new_end_date), new_end_date)
try:
    slice=fablib.get_slice(name=slice_name)
    slice.renew('2023-05-31 18:04:26 +0000') # Give the new lease end date and time of the slice. One can increase it by 7 days from the day of creation of the slice/cluster. 
    
    #print(f"Lease End (UTC)        : {slice.get_lease_end()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Run this cell ONLY to observe the new lease time.

slice_name='cluster_exp' # Give the slice/cluster name that you just extended. 
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Run this cell ONLY to observe the new lease time.

slice_name='cluster_gpu' # Give the slice/cluster name that you just extended. 
try:
    slice = fablib.get_slice(name=slice_name)
    gpu_name="NVIDIA"
    for node in slice.get_nodes():
        #stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') # if you run the command twice the back up will be overwritten, a conditional block should be written
        stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
        if gpu_name in stdout_gpu:
            stdout_ip, _=node.execute('hostname -I')
            ip=stdout_ip.split(" ")[1]
            print(ip)
except Exception as e:
    print(f"Exception: {e}")

## Do not bother about the code below !!! ;-)

In [ ]:
# Do not Run this cell

storage_name = 'gaf-storage1'

node1=slice.get_node(name='Node1')
node1.add_component(model='NVME_P4510',name='gaf-storage')

In [ ]:
slice_name='cluster_gpu_2'

try:
    slice=fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Do not run this cell, only for experimentation.

try:
    node = slice.get_node(name='Node4')
    print(f"{node}")
    storage = node.get_storage(name='gaf-storage')
    print(f"{storage}")
    print(f"Storage Device Name: {storage.get_device_name()}")
    #print("Mounting the storage volume")
    #stdout, stderr = node.execute(f"sudo mkdir -p /mnt/{storage_name};"
                                 # f"sudo mount {storage.get_device_name()} /mnt/{storage_name}")
    print(stdout)
    print(stderr)
except Exception as e:
    print(f"Exception: {e}")



In [ ]:
hosts=[]
import random
randomList=[]

while True:
    r=random.randint(1,9)
    
    if r not in randomList:
        randomList.append(r)
    if len(randomList) == 8:
        break
        
for i in range(1,num_nodes+1):
    hosts.append("{0}-w{1}.fabric-testbed.net".format(site.lower(),randomList[i-1]))
    
print(hosts)  

In [ ]:
try:
    node = slice.get_node('Node3') 
    #node.show()

    nvme1 = node.get_component(nvme_names[1])
    nvme1.show()
    nvme1.configure_nvme()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
nvme_names[1]